In [1]:
import math
import pandas as pd
from statsmodels.stats.power import tt_ind_solve_power 

import numpy as np
import statsmodels.stats.power as smp
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

# Рассчет размера выборки





Введите базовую конверсию **basic_conversion** - в %

Желаемый уровень мощности **power** - в % (Больше 80%)

**Alpha**  - (не может быть больше 0.05)

А теперь устанавливаем для каких MDE надо рассчитать размер выборки:

**begin** - в % (то есть, рассчитываем на повышение конверсии на минимум 10%)

**to** - в % (то есть, рассчитываем на повышение конверсии на максимум 50%)

**step** - в% (нас интересует размер выборки, если планирует увеличить конверсию на 10%, 15%, 20%...50% (с 5% шагом)

In [2]:
basic_conversion =  52
power =  80
alpha = 0.05
begin =  5
to =  30
step =  5
res=pd.DataFrame()
for i in range(begin, to+1, step):
    p_y = basic_conversion/100*(1+i/100)
    h = 2*math.asin(np.sqrt(basic_conversion/100)) - 2*math.asin(np.sqrt(p_y))
    sample_size=smp.zt_ind_solve_power(effect_size = h, alpha = alpha, power = power/100, alternative='two-sided')
    to_insert = {
         "mde": int(i),
        "sample_size_variation": int(sample_size),
        "sample_size_test": int(sample_size)*2
       
    }
    
    res = res.append(to_insert, ignore_index=True)

In [3]:
res.set_index('mde').T

mde,5.0,10.0,15.0,20.0,25.0,30.0
sample_size_variation,5778.0,1437.0,634.0,353.0,224.0,153.0
sample_size_test,11556.0,2874.0,1268.0,706.0,448.0,306.0


# Рассчет мощности исходя из желаемого лифта и sample size (t-student)

### Если у вас биномиальное распределение, поставьте в поле std 0

### В случае, если у вас не биномиальное распределение, а близко к нормальному вам нужно рассчитать стандартное отклонение. Или просто введите в поле std ваше значение.


Рассчитать можно по формуле ниже, просто подставьте массив  с вашими данными. Или переходите к следующему шагу


In [4]:
data=pd.Series(df.column)
data.std()

NameError: name 'df' is not defined

Введите базовую конверсию **basic_conversion** - в десятичном формате (то есть. если конверсия 2% - вводите 0.02)

**Alpha** - (не может быть больше 0.05)

Определяем, в каких пределах будем считать мощность:

**Min_sample** -  минимальный размер выборки для теста

**Max_sample** -  максимальный размер выборки для теста

**step_sample** - шаг для выборки

**Min_lift** -  минимальный лифт

**Max_lift** -  максимальный лифт

**step_lift** - шаг для лифта

**std** - стандартное отклонение (в случае биномиального распределения поставьте 0 либо свое рассчитанное значение)

In [8]:
basic_conversion = 0.17
alpha = 0.05
min_sample =  1000
max_sample = 3000
step_sample = 500
min_lift = 10
max_lift =  25
step_lift = 1
std = 0
res2=pd.DataFrame()

if std==0:
  se = np.sqrt(basic_conversion*(1-basic_conversion))
else: 
  se=std
for i in range(min_sample, max_sample+1, step_sample):
    for lift in range(min_lift, max_lift+1, step_lift):
        effect_size = basic_conversion/se * (lift/100)
        power=tt_ind_solve_power(effect_size=effect_size, alpha=alpha,  nobs1=i/2)
    
        to_insert = {
            "sample_size_test": round(i),
            "power": power,
            "lift": lift
    }
        res2 = res2.append(to_insert, ignore_index=True)

In [9]:
res3=pd.pivot_table(res2, values='power', index=['sample_size_test'],
                    columns=['lift'], aggfunc=np.sum)




In [10]:
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'lightseagreen' if val >0.9 else 'limegreen' if val >0.8  else 'black'
    return 'color: %s' % color
res4 = res3.style.applymap(color_negative_red)
res4

lift,10.0,11.0,12.0,13.0,14.0,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0
sample_size_test,,,,,,,,,,,,,,,,
1000.0,0.110290,0.123293,0.137627,0.153292,0.170283,0.188582,0.208164,0.228988,0.251004,0.274147,0.298340,0.323493,0.349505,0.376262,0.403643,0.431515
1500.0,0.141439,0.161224,0.182987,0.206688,0.232258,0.259602,0.288596,0.319086,0.350892,0.383807,0.417603,0.452033,0.486838,0.521750,0.556501,0.590825
2000.0,0.172926,0.199468,0.228533,0.259983,0.293630,0.329234,0.366506,0.405115,0.444692,0.484843,0.525157,0.565222,0.604630,0.642999,0.679975,0.715244
2500.0,0.204557,0.237712,0.273783,0.312482,0.353441,0.396218,0.440314,0.485182,0.530250,0.574940,0.618691,0.660978,0.701328,0.739342,0.774700,0.807169
3000.0,0.236160,0.275684,0.318339,0.363631,0.410967,0.459668,0.509000,0.558207,0.606536,0.653279,0.697799,0.739556,0.778125,0.813207,0.844631,0.872351
